# Testando paralelização em GPUS através da biblioteca Numba

## Multiplicando vetores (CPU)

### Declarando a função

In [14]:
import numpy as np

def mulVectorsCPU(a:list[np.float64], b:list[np.float64]):
    if a.size != b.size: raise RuntimeError('mulVectorsCPU() expects two vectors (lists) of equal length!')

    c = [] # Vector to be returned
    for i in range(a.size): c.append(a[i] * b[i])
    return c

### Inicializando vetores para testes

Inicializando dois vetores **`a`** e **`b`** com **1,000,000,000 (um bilhão) de floats randomizados** (no **intervalo [-1000, +1000]**)

In [20]:
N = 10000000
MIN_VAL = -100.0
MAX_VAL = 100.0

# Initializing vectors with zeroes (with default dtype -> np.float64)
a = np.zeros(N)
b = np.zeros(N)

for i in range(N):
    a[i] = np.random.uniform(MIN_VAL, MAX_VAL)
    b[i] = np.random.uniform(MIN_VAL, MAX_VAL)

print(f'a[:5] = {a[:5]}; a[-5:] = {a[-5:]}')
print(f'b[:5] = {b[:5]}; b[-5:] = {b[-5:]}\n')

a[:5] = [-97.55206547 -76.982899   -42.93920612  12.21265777  93.61409053]; a[-5:] = [-35.03425009 -94.20764148  15.73056993 -69.4473211  -79.7350862 ]
b[:5] = [-27.31446483  45.45453459 -61.73734635 -56.627657    32.27106207]; b[-5:] = [-67.12440957 -68.18581055 -67.94416229  93.81666111 -91.13087531]

